In [3]:
%matplotlib qt5

## 跑通最小 demo

In [5]:
import kineticstoolkit.lab as ktk

filename = ktk.doc.download("kinematics_tennis_serve.c3d")
c3d_contents = ktk.read_c3d(filename)
markers = c3d_contents["Points"]


# Non-interactive, inline backend
%matplotlib inline

# Create the player and set a viewpoint
player = ktk.Player(
    markers,
    up="z",
    zoom=1.5,
    azimuth=0.8,
    elevation=0.16,
    translation=(-0.2, -1.0),
)

# Create an html video to be included in the Jupyter Notebook
player.to_html5(
    start_time=6,
    stop_time=8
)

UserWarning [/Users/wangshuaibo/anaconda3/envs/kineticstoolkit/lib/python3.11/site-packages/kineticstoolkit/player.py:361] frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.


: 

In [1]:
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## 练习点、向量以及坐标的转换
参考链接：https://kineticstoolkit.uqam.ca/doc/geometry_transform_moving_coordinates.html

In [84]:

# 练习点、向量以及坐标系的转换
angle_rad = np.deg2rad(30)
R = np.array([
    [np.cos(angle_rad), -np.sin(angle_rad),0],
    [np.sin(angle_rad), np.cos(angle_rad),0],
    [0,0,1],
    [0,0,0]
])
# 计算变化矩阵
T = np.concatenate((R,P),axis=1) # 按照列进行拼接

# 计算点的变化
# 点的变化即有旋转又有平移
p_initial = np.array([[10],
                      [0],
                      [0],
                      [1]])
p_transformed = np.dot(T, p_initial)
print("点变换后的坐标：\n",np.round(p_transformed,2))

print("\n")
# 计算向量的变化
# 向量的变化只有旋转，没有位置
# 向量最后一位是 0 和坐标相乘就会变为 0 
# 向量和点的坐标位置是一样的，但就是最后一位数
vector_initial = np.array([[10],
                           [0],
                           [0],
                           [0]])
vector_transformed = np.dot(T,vector_initial)
print("向量变化后的坐标：\n", np.round(vector_transformed,2))

frame_initial = np.array([
    [1,0,0,10],
    [0,1,0,0],
    [0,1,0,0],
    [0,0,0,1]
])
frame_transformed = np.dot(T,frame_initial)
print("坐标轴变化后的坐标：\n",np.round(frame_transformed,2))

点变换后的坐标：
 [[10.66]
 [ 5.  ]
 [ 0.  ]
 [ 1.  ]]


向量变化后的坐标：
 [[8.66]
 [5.  ]
 [0.  ]
 [0.  ]]
坐标轴变化后的坐标：
 [[ 0.87 -0.5   0.   10.66]
 [ 0.5   0.87  0.    5.  ]
 [ 0.    1.    0.    0.  ]
 [ 0.    0.    0.    1.  ]]


In [94]:
# 使用 kinetics 的方法
import kineticstoolkit.lab as ktk
T = ktk.geometry.create_transforms(
    seq="z",
    angles = [30],
    translations=[[2,0,0]],
    degrees=True
)
# 知道角度，就知道旋转矩阵了
print("变换矩阵为：\n",T) 



变换矩阵为：
 [[[ 0.8660254 -0.5        0.         2.       ]
  [ 0.5        0.8660254  0.         0.       ]
  [ 0.         0.         1.         0.       ]
  [ 0.         0.         0.         1.       ]]]


array([[10.66025404,  5.        ,  0.        ,  1.        ]])

In [97]:
# 变换点
# ktk 不支持array，内部会将列表转换为矩阵
ktk.geometry.matmul(T, [[10, 0 , 0, 1]])


array([[8.66025404, 5.        , 0.        , 0.        ]])

In [ ]:
# 变换向量
ktk.geometry.matmul(T,[[10, 0, 0, 0]])

In [99]:
# 变换坐标轴
# 注意这里的列表输入形式，行数为 n+1 个逗号
ktk.geometry.matmul(
    T,[[[1, 0, 0, 10],[0,1,0,0],[0,0,1,0],[0,0,0,1]]]
)

array([[[ 0.8660254 , -0.5       ,  0.        , 10.66025404],
        [ 0.5       ,  0.8660254 ,  0.        ,  5.        ],
        [ 0.        ,  0.        ,  1.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]])

## 东北天到西北天
缩写介绍：
MTi 的默认地球固定局部坐标系 (L)（在正常输出模式下）是东-北-天 (East-North-Up, ENU) 坐标系。 MTi 可以轻松设置为使用另外两个参考坐标系。 它们是 NWU（北-西-天, North West Up）和 NED（北-东-地, North East Down）。

详细内容参见 keynote 中绘制的图

In [15]:
# 理解从东北天到北西天的逆矩阵
import numpy as np
rad_90 = np.deg2rad(90)

# 北西天的坐标转换为东北天的旋转矩阵推导过程
# https://kineticstoolkit.uqam.ca/doc/geometry_basics.html

R_NWU_to_ENU_frame = np.array([
    [np.cos(rad_90), -np.sin(rad_90), 0],
    [np.sin(rad_90), np.cos(rad_90), 0],
    [0,0,1]
])
print("北西天坐标系转换到东北天\n",
      np.round(R_NWU_to_ENU_frame,2))

# 东北天➡️北西天的旋转矩阵 是北西天➡️东北天旋转矩阵的逆矩阵
# 结果和Xsens 官网结果一致 https://movella.my.site.com/XsensKnowledgebase/s/article/Changing-or-Resetting-the-MTi-reference-co-ordinate-systems-1605869706643?language=zh_CN
R_ENU_to_NMU_frame = np.linalg.inv(R_NWU_to_ENU_frame)
print("东北天坐标系转换为北西天:\n",
      np.round(R_ENU_to_NMU_frame,2))


北西天坐标系转换到东北天
 [[ 0. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]]
东北天坐标系转换为北西天:
 [[ 0.  1.  0.]
 [-1.  0. -0.]
 [ 0.  0.  1.]]
